In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np

In [2]:
totals_words = 10000#只使用最常用的前10000个单词
embedding_len = 100#每个单词的维度为100
max_review_len = 80#每个句子的长度
batch_size = 128
(x_train,y_train),(x_val,y_val) = keras.datasets.imdb.load_data(num_words=totals_words)#只使用最常用的10000个单词，其他单词标记为未知
x_train = keras.preprocessing.sequence.pad_sequences(x_train,maxlen=max_review_len)#设置每个句子的长度为80
x_val = keras.preprocessing.sequence.pad_sequences(x_val,maxlen=max_review_len)
db_train = tf.data.Dataset.from_tensor_slices((x_train,y_train)).shuffle(10000).batch(batch_size,drop_remainder=True)
db_val = tf.data.Dataset.from_tensor_slices((x_val,y_val)).shuffle(1000).batch(batch_size,drop_remainder=True)#最后一个batch长度不够将其drop

db_iter = iter(db_train)
sample = next(db_iter)
print(x_train.shape,y_train.shape,tf.reduce_max(y_train),tf.reduce_min(y_train))
print(x_val.shape,y_val.shape)
print(sample[0].shape)
print(sample[1].shape)

(25000, 80) (25000,) tf.Tensor(1, shape=(), dtype=int64) tf.Tensor(0, shape=(), dtype=int64)
(25000, 80) (25000,)
(128, 80)
(128,)


In [7]:
class MyRnn(keras.Model):
    def __init__(self,units):
        super(MyRnn,self).__init__()
        #[b,80]=>[b,80,100]
        self.embedding = layers.Embedding(input_dim=totals_words,output_dim=embedding_len,input_length=max_review_len)
        
        #[b,80]=>[b,units]
        self.rnn_cell0 = layers.SimpleRNN(units,return_sequences=True,dropout=0.5,unroll=True)#返回输出序列中的最后一个输出
        self.rnn_cell1 = layers.SimpleRNN(units,dropout=0.5,unroll=True)#unroll 是否展开网络
        #[b,units]=>[b,1]
        self.fc = layers.Dense(1)
        
    def call(self,inputs,training=None):
        #x = [b,80]
        x = inputs
        #[b,80]=>[b,80,100]
        x = self.embedding(x)
        #[b,80]=>[b,64]
        x = self.rnn_cell0(x)
        x = self.rnn_cell1(x)
        #[b,64]=>[b,1]
        out = self.fc(x)
        #p(y is positive|x)
        pred = tf.sigmoid(out)
        
        return pred

In [8]:
def main():
    units = 64
    model = MyRnn(units)
    model.compile(optimizer=keras.optimizers.Adam(0.0001),loss=tf.losses.BinaryCrossentropy(),metrics=['accuracy'])
    model.fit(db_train,epochs=10,validation_data=db_val)
main()

Epoch 1/10
195/195 [==============================] - 27s 137ms/step - loss: 0.6938 - accuracy: 0.5219 - val_loss: 0.6073 - val_accuracy: 0.6761
Epoch 2/10
195/195 [==============================] - 11s 57ms/step - loss: 0.5697 - accuracy: 0.6754 - val_loss: 0.4656 - val_accuracy: 0.7819
Epoch 3/10
195/195 [==============================] - 11s 57ms/step - loss: 0.4654 - accuracy: 0.7722 - val_loss: 0.4204 - val_accuracy: 0.8103
Epoch 4/10
195/195 [==============================] - 11s 57ms/step - loss: 0.4054 - accuracy: 0.8144 - val_loss: 0.4106 - val_accuracy: 0.8218
Epoch 5/10
195/195 [==============================] - 11s 57ms/step - loss: 0.3675 - accuracy: 0.8394 - val_loss: 0.4148 - val_accuracy: 0.8242
Epoch 6/10
195/195 [==============================] - 11s 56ms/step - loss: 0.3408 - accuracy: 0.8508 - val_loss: 0.4364 - val_accuracy: 0.8244
Epoch 7/10
195/195 [==============================] - 11s 55ms/step - loss: 0.3163 - accuracy: 0.8672 - val_loss: 0.4204 - val_accuracy